In [ ]:
from keras.applications import inception_v3
from keras import backend as K
import sys
K.set_learning_phase(0)
model=inception_v3.InceptionV3(weights='imagenet',include_top=False)

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
sys.setrecursionlimit(1500)

In [ ]:
layer_contribuions={
    'mixed2':0.2,
    'mixed3':3.,
    'mixed4':2.,
    'mixed5':1.5,

}

In [ ]:
layer_dict=dict([(layer.name,layer)for layer in model.layers])

loss=K.variable(0.)

for layer_name in layer_contribuions:
    coeff=layer_contribuions[layer_name]
    activation=layer_dict[layer_name].output

    scaling=K.prod(K.cast(K.shape(activation),'float32'))
    loss.assign_add( coeff*K.sum(K.square(activation[:,2:-2,2:-2,:]))/scaling)

In [ ]:
dream=model.input
grads=K.gradients(loss,dream)[0]
grads/=K.maximum(K.mean(K.abs(grads)),1e-7)

outputs=[loss,grads]
fetch_loss_and_grads=K.function([dream],outputs)

def eval_loss_and_grads(x):
    outs=fetch_loss_and_grads(x)
    loss_value=outs[0]
    grad_value=outs[1]
    return loss_value,grad_value

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values=eval_loss_and_grads(x)

        if max_loss is not None and loss_value>max_loss:
            break
        print('Wartość straty ',i,' : ', loss_value)
        x+=step*grad_values
    return x



In [ ]:
import numpy as np

step=0.01
num_octave=3
octave_scale=1.4
interations=20

max_loss=10

base_image_path='_'
img=preprocess_image(base_image_path)

